# Problema do Empacotamento

Problema bastante importante na área de logistica, mais especficamente na área de armazenagem.

Objetivo do problema é gaurdar todos os itens em caixas minimizando a quantidade de caixas necessárias.

**Observação** Os itens não podem ser fracionados.

Índices:
- i = 1, ..., m itens;
- j = 1, ..., n caixas;

Parâmetroas:
- $w_i$ peso do item i;
- $C$ capacidade das caixas;

Limitante inferior = $\left[\frac{\sum_{i=1}^{n}w_i}{C}\right]$. Corresponde uma estimativa de quanto que é o melhor caso possível para solução ótima.

Limitante superior obtém pelo algoritmo ``next fit``.


Variáveis de Decisão (todas binárias)
- $x_{ij}$ indica se o item i será colocado na caixa j;
- $y_j$ indica se a caixa j foi usada na solução;

#### Função Objetivo

$$
\min\left[ \sum_{j=1}^{n}y_j \right]
$$

#### Função Restrição:

restrição na capacidade da caixa
$$
\sum_{i=1}^{m}w_ix_{ij} \leq C y_{j}, \;\; \forall j=1, ..., n
$$

todos os itens devem estar em uma caixa
$$
\sum_{j=1}^{n}x_{ij} = 1, \;\; \forall i=1, ..., m
$$

In [1]:
import gurobipy as gp

In [2]:
def calcula_qtd_caixas(itens, pesos, capacidade):
    """
    Calcula a quantidade de caixas usando o algoritmo ``first fit``
    """
    qtd_caixas = 1
    capacidade_usada = pesos[itens[0]]
    for item in itens[1:]:
        capacidade_usada += pesos[item]
        if capacidade_usada > capacidade:
            qtd_caixas += 1
            capacidade_usada = pesos[item]
    return qtd_caixas

def read_instancia(arq):
    with open(arq, 'r') as f:
        lines = f.readlines()
        # Ler a quantidade de itens e a capacidade da caixa
        qtd_itens, capacidade = [int(x) for x in lines[0].strip().split(' ')]
        # Gera os rótulos dos itens
        itens = [f'Item_{i}' for i in range(1, qtd_itens+1)]
        # Dicionário de pesos dos itens
        pesos = {itens[i]:int(peso) for i, peso in enumerate(lines[1].strip().split(' '))}
        # Rótulo das caixas
        qtd_caixas = calcula_qtd_caixas(itens, pesos, capacidade)
        caixas = [f'Caixa_{j}' for j in range(1,qtd_caixas+1)]
    return capacidade, itens, caixas, pesos

In [4]:
def solver(arq):
    # Lê os dados da instância
    capacidade, itens, caixas, pesos = read_instancia(arq)
    
    # Modelo
    m = gp.Model()

    # Variáveis de decisão
    x = m.addVars(itens, caixas, vtype=gp.GRB.BINARY)
    y = m.addVars(caixas, vtype=gp.GRB.BINARY)

    # Função Objetivo
    m.setObjective(
        gp.quicksum(y[j] for j in caixas),
        sense=gp.GRB.MINIMIZE
    )

    # Restrição de todos os itens em uma caixa
    c1 = m.addConstrs(
        gp.quicksum(x[i,j] for j in caixas) == 1 for i in itens
    )

    # Restrição de capacidade
    c2 = m.addConstrs(
        gp.quicksum(pesos[i]*x[i,j] for i in itens) <= capacidade*y[j] for j in caixas
    )

    # Executar o modelo
    m.setParam(gp.GRB.Param.OutputFlag, 0)   # Não imprimir o relatório.
    m.setParam(gp.GRB.Param.TimeLimit, 5)    # Tempo limite de execução do modelo com apenas '5 segundos'.    
    m.optimize()
    
    return m.objVal, m.Status

In [5]:
instancia = 'Empacotamento/inst_{:03d}.txt'
solucao = list()
for i in range(1,101):
    arq_inst = instancia.format(i)
    caixas, status = solver(arq_inst)
    solucao.append((caixas, status))
    print('Instância', i)
    #print(f"O solver encontrou uma solução com {caixas} caixas")

Using license file /home/octavio/gurobi.lic
Academic license - for non-commercial use only
Instância 1
Instância 2
Instância 3
Instância 4
Instância 5
Instância 6
Instância 7
Instância 8
Instância 9
Instância 10
Instância 11
Instância 12
Instância 13
Instância 14
Instância 15
Instância 16
Instância 17
Instância 18
Instância 19
Instância 20
Instância 21
Instância 22
Instância 23
Instância 24
Instância 25
Instância 26
Instância 27
Instância 28
Instância 29
Instância 30
Instância 31
Instância 32
Instância 33
Instância 34
Instância 35
Instância 36
Instância 37
Instância 38
Instância 39
Instância 40
Instância 41
Instância 42
Instância 43
Instância 44
Instância 45
Instância 46
Instância 47
Instância 48
Instância 49
Instância 50
Instância 51
Instância 52
Instância 53
Instância 54
Instância 55
Instância 56
Instância 57
Instância 58
Instância 59
Instância 60
Instância 61
Instância 62
Instância 63
Instância 64
Instância 65
Instância 66
Instância 67
Instância 68
Instância 69
Instância 70
Instânci

In [27]:
for idx, sol in enumerate(solucao, 1):
    caixa, status = sol
    if status == 2:
        print(f"Instânica {idx:03d}: Encontrou uma solução com {caixa} caixas - SOLUÇÃO ÓTIMA.")
    else:
        print(f"Instânica {idx:03d}: Encontrou uma solução com {caixa} caixas - TEMPO LIMITE.")

Instânica 001: Encontrou uma solução com 23.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 002: Encontrou uma solução com 75.0 caixas - TEMPO LIMITE.
Instânica 003: Encontrou uma solução com 19.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 004: Encontrou uma solução com 52.0 caixas - TEMPO LIMITE.
Instânica 005: Encontrou uma solução com 31.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 006: Encontrou uma solução com 22.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 007: Encontrou uma solução com 35.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 008: Encontrou uma solução com 35.0 caixas - TEMPO LIMITE.
Instânica 009: Encontrou uma solução com 65.0 caixas - TEMPO LIMITE.
Instânica 010: Encontrou uma solução com 59.0 caixas - TEMPO LIMITE.
Instânica 011: Encontrou uma solução com 52.0 caixas - TEMPO LIMITE.
Instânica 012: Encontrou uma solução com 13.0 caixas - TEMPO LIMITE.
Instânica 013: Encontrou uma solução com 13.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 014: Encontrou uma solução com 19.0 caixas - SOLUÇÃO ÓTIMA.
Instânica 015: Encontrou um